In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

In [2]:
# This is the number the step size is divided by (aka larger d yields smaller bin size)
# Step sizes used: [400, 200, 100, 50, 25, 18, 12.5]
d = 18

destdir = 'BinnedCsvs_d{}'.format(d)
csvs = [ os.path.join(destdir,f) for f in os.listdir(destdir) ]

In [4]:
def main_check(csv, reduced=False):
    '''Check csv, if reduced == True, check csv when it has been reduced to the top 10 stations'''
    try:
        csv_passes = True

        # Read in the csv (Created by GHCN_Binned.ipynb)
        df = pd.read_csv(csv, index_col=0)
        # Only want TMAX and TMIN elements
        # df = df[df.element.isin(['TMAX', 'TMIN'])]

        # New binning used different variable names, for compatability
        df.reset_index(0, inplace=True)
        df.columns = ['id', 'date', 'element', 'value']

        # Use datetime for easy indexing
        df['date'] = pd.to_datetime(df['date'])
        
        
        ################### REDUCE #########################
        # If we want to perform the checks using only the top 10 most frequent stations in each bin
        if reduced == True:
            station_count_df = df.groupby('id')['id'].agg({'station_count':'count'})
            station_list_sorted = list(station_count_df.sort_values('station_count', ascending=False).index)
            top10_stations = station_list_sorted[:10]

            df = df[df['id'].isin(top10_stations)]
        ####################################################

        # Convert table from long format to wide
        df_wide = df.pivot_table(index=['id', 'date'], columns=['element'], values='value')

        # Convert temperature readings to degrees C and precipitation readings to mm 
        # (not needed for checking)
        # df_wide['TMAX'] /= 10
        # df_wide['TMIN'] /= 10

        df_wide.reset_index(0, inplace=True)

        # Check 1: make sure the datafile has data for years 2005-2015 and correct columns
        # Returns True if the data passes the check
        def check_1():
            a = set(df_wide.columns) >= {'TMAX', 'TMIN', 'id'}
            b = set(np.arange(2005,2016)) <= set(df_wide.index.year)
            return a and b
        
        # Update csv_passes to reflect result from the first check
        csv_passes = check_1()
    
        # If the first check is failed, return
        if csv_passes == False:
            return (csv, csv_passes)
    
        #########################################################################

        # Number of years to aggregate over that will be compared against final_year
        num_years = 10

        # The final year that will be used to check for broken records
        final_year = 2015

        #########################################################################

        # Split the data into the last year and the previous years
        previousyears = df_wide[str(final_year-num_years):str(final_year-1)]
        lastyear = df_wide[str(final_year)]

        # Calculate previous years' record highs and lows
        previousyears_maxmin = pd.groupby(previousyears, by=[previousyears.index.month,previousyears.index.day]).agg({'TMAX':'max', 'TMIN':'min'})

        # Calculate the last year's record highs and lows
        lastyear_maxmin = pd.groupby(lastyear, by=[lastyear.index.month,lastyear.index.day]).agg({'TMAX':'max', 'TMIN':'min'})


        # Only do check 2 if check 1 passes
        # check 2: length of lastyear_maxmin['TMAX'] lastyear_maxmin['TMIN'] previousyears_maxmin['TMAX'] 
        # and previousyears_maxmin['TMIN'] should be 365 or 366.
        # Returns True if the data passes the check
        def check_2():
            # Get lengths
            n1 = lastyear_maxmin['TMAX'].count()
            n2 = lastyear_maxmin['TMIN'].count()
            n3 = previousyears_maxmin['TMAX'].count()
            n4 = previousyears_maxmin['TMIN'].count()
            # Check that lengths are 365 or 366
            return set([n1, n2, n3, n4]) <= set([365, 366])


        # Only do check 3 if check 2 passes
        # check 3: There are at least x tmin outliers + tmax outliers in the last year
        # Returns True if the data passes the check
        def check_3(x=1):
            # Deal with leap years
            leap_day_missing = False

            try:
                leap_day_missing = False if previousyears_maxmin.xs([2,29]).count() == 2 else True
            except:
                leap_day_missing = True

            if not leap_day_missing:
                previousyears_WLY = previousyears_maxmin.loc[(previousyears_maxmin.index.get_level_values(0) != 2) | 
                                                             (previousyears_maxmin.index.get_level_values(1) != 29)]
            else:
                previousyears_WLY = previousyears_maxmin

            # Find record highs and lows (outliers)
            record_high_lastyear = lastyear_maxmin['TMAX'].where(lastyear_maxmin['TMAX'] >= previousyears_WLY['TMAX'])
            record_low_lastyear = lastyear_maxmin['TMIN'].where(lastyear_maxmin['TMIN'] <= previousyears_WLY['TMIN'])

            total_number_of_outliers = record_high_lastyear.count()+record_low_lastyear.count()

            return total_number_of_outliers >= x


        csv_passes = check_2()

        if csv_passes == False:
            return (csv, csv_passes)
        else:
            csv_passes = check_3()
            return (csv, csv_passes)
    except:
        return '{} failed!'.format(csv)

In [5]:
# Iterate through all the csvs and test them using main_check
results = []
for csv in tqdm(csvs):
    results.append(main_check(csv))

100%|██████████| 104/104 [04:32<00:00,  5.91s/it]


In [6]:
# These are the csvs that raised an error
error_csvs = []
for i in results:
    if type(i) != tuple:
        csv_name = i.split()[0]
        error_csvs.append(csv_name)
print('There is/are {} csv/s that produced an error.'.format(len(error_csvs)))

There is/are 0 csv/s that produced an error.


In [7]:
# These are the csvs that failed (should be deleted)
failed_csvs = []
for i in results:
    if i[1] == False:
        failed_csvs.append(i[0])
print('There are {} csvs that failed.'.format(len(failed_csvs)))

There are 46 csvs that failed.


In [8]:
# These are the csvs that passed (should be used for the assignment)
passed_csvs = []
for i in results:
    if i[1] == True:
        passed_csvs.append(i[0])
print('There are {} csvs that passed.'.format(len(passed_csvs)))

There are 58 csvs that passed.


In [9]:
# Sanity check
len(failed_csvs) + len(passed_csvs) + len(error_csvs) == len(csvs)

True

In [10]:
# check what stations are in the passed_csvs
passed_stations = []
for csv in passed_csvs:
    passed_df = pd.read_csv(csv, index_col=0)
    passed_stations += list(passed_df.index.unique())

print('There are {} stations left!'.format(len(passed_stations)))

In [12]:
# Save the stations that passed to csv
#pd.Series(passed_stations).to_csv('passed_stations_d{}.csv'.format(d))